# instructGOOSE

> Implementation of Reinforcement Learning from Human Feedback (RLHF)

This file will become your README and also the index of your documentation.

## Install

```sh
pip install instructGOOSE
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2